<a href="https://colab.research.google.com/github/bluealphanumeric9/Quantitative-Finance/blob/main/Master_Thesis_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The following notebook contains the relevant coding for the Master Thesis

##Part 1

The following code produces the accurate and efficient  pricing and approximation of Greeks using Carr-Madan integrand
and differentiation under the Carr-Madan integral
It produces FFT prices and FFT Greeks at once getting the Greeks for an entire strike grid with $O(N log N)$ complexity for many strike which is far cheaper than per-strike quadrature or finite-difference bumping

In [ ]:
import numpy as np


# ==============================
# 1) Stable Heston characteristic function (Lord–Kahl “little trap”)
# ==============================
def heston_charfunc_lord_kahl(u, T, v0, kappa, theta, sigma, rho, r, q, return_CD=False):
    """
    phi(u) = exp(C(u,T) + D(u,T) * v0), stable (Lord & Kahl).
    If return_CD=True, also returns (C, D).
    """
    iu = 1j * u
    a  = kappa * theta
    b  = kappa - rho * sigma * iu
    d  = np.sqrt(b*b + (iu + u*u) * sigma * sigma)
    g  = (b - d) / (b + d)
    e  = np.exp(-d * T)
    C  = (r - q) * iu * T + (a / (sigma*sigma)) * ((b - d) * T - 2.0 * np.log((1 - g*e) / (1 - g)))
    D  = ((b - d) / (sigma*sigma)) * ((1 - e) / (1 - g*e))
    phi = np.exp(C + D * v0)
    if return_CD:
        return phi, C, D
    return phi

def phi_logS_shifted(z, S0, T, v0, kappa, theta, sigma, rho, r, q, return_CD=False):
    """
    Characteristic function of log S_T evaluated at shifted argument z = u - i(α+1).
    Includes S0^{(α+1)} * e^{i u log S0} factor implicitly via exp(i z log S0).
    """
    # exp(i z log S0)
    S_factor = np.exp(1j * z * np.log(S0))
    if return_CD:
        phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
        return S_factor * phi, C, D
    else:
        phi = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q)
        return S_factor * phi

# ==============================
# 2) Reference (single-strike) price and Greeks via analytic integrands
# ==============================
def cm_price_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                     alpha=1.5, N=2**14, u_max=200.0):
    """
    Carr–Madan damped price at a single strike (high-accuracy quadrature).
    """
    k  = np.log(K)
    du = u_max / N
    u  = np.arange(N) * du
    beta = alpha + 1.0
    z = u - 1j * beta

    phi = phi_logS_shifted(z, S0, T, v0, kappa, theta, sigma, rho, r, q)
    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    psi = np.exp(-r*T) * phi / denom
    integrand = np.exp(-1j * u * k) * psi  # keep complex

    # Simpson weights (complex-safe)
    if (N-1) % 2 == 0:
        w = np.ones(N); w[0]=1.0; w[-1]=1.0; w[1:-1:2]=4.0; w[2:-1:2]=2.0
        integral = (du/3.0) * np.sum(w * integrand)
    else:
        w = np.ones(N); w[0]=0.5; w[-1]=0.5
        integral = du * np.sum(w * integrand)

    C = (np.exp(-alpha * k) / np.pi) * integral
    return float(np.real(C))

def cm_greeks_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                      alpha=1.5, N=2**14, u_max=200.0):
    """
    Reference Greeks via analytic differentiation under the integral:
      Delta: multiply integrand by (i z / S0)
      Gamma: multiply integrand by (-(i z + z^2)/S0^2)
      Vega (w.r.t. v0): multiply integrand by D(z), where phi = exp(C + D v0)
    """
    k  = np.log(K)
    du = u_max / N
    u  = np.arange(N) * du
    beta = alpha + 1.0
    z = u - 1j * beta

    # phi, C, D at shifted argument
    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    # include S0 factor: exp(i z log S0)
    S_term = np.exp(1j * z * np.log(S0))
    phiS   = S_term * phi

    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    base  = np.exp(-r*T) * phiS / denom
    phase = np.exp(-1j * u * k)

    # Price integrand (for completeness; not needed here)
    # price_int = phase * base

    # Delta integrand: (i z / S0) * (price integrand w/o the final exp(-α k)/π factor)
    delta_int = phase * base * (1j * z / S0)

    # Gamma integrand: (-(i z + z^2)/S0^2) * base * phase
    gamma_int = phase * base * (-(1j * z + z*z) / (S0*S0))

    # Vega(v0) integrand: D(z) * base * phase
    vega_int  = phase * base * D

    # Simpson weights (complex-safe)
    if N % 2 == 0:
        w = np.ones(N); w[0]=1.0; w[-1]=1.0; w[1:-1:2]=4.0; w[2:-1:2]=2.0
        W = (du/3.0) * w
    else:
        w = np.ones(N); w[0]=0.5; w[-1]=0.5
        W = du * w

    factor = np.exp(-alpha * k) / np.pi

    Delta = factor * np.sum(W * delta_int)
    Gamma = factor * np.sum(W * gamma_int)
    Vega  = factor * np.sum(W * vega_int)

    # Greeks must be real for real inputs
    return float(np.real(Delta)), float(np.real(Gamma)), float(np.real(Vega))

# ==============================
# 3) FFT (Carr–Madan) grid: price and Greeks via analytic integrands
# ==============================
def cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                    alpha=1.5, N=2**12, eta=0.25, k0=None):
    """
    Return (k_grid, Price, Delta, Gamma, Vega_v0) arrays (length N)
    using a single FFT for each integrand flavor.
    """
    if k0 is None:
        k0 = np.log(S0) - 3.0

    m = np.arange(N)
    u = m * eta
    beta = alpha + 1.0
    z = u - 1j * beta

    # phi, C, D at shifted argument
    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    S_term = np.exp(1j * z * np.log(S0))
    phiS   = S_term * phi

    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    base  = np.exp(-r*T) * phiS / denom

    # The four frequency-domain inputs (price + 3 Greeks)
    F_price = base
    F_delta = base * (1j * z / S0)
    F_gamma = base * (-(1j * z + z*z) / (S0*S0))
    F_vega  = base * D

    # Simpson weights in frequency
    w = np.ones(N); w[0]=1.0; w[-1]=1.0; w[1:-1:2]=4.0; w[2:-1:2]=2.0
    w *= (eta / 3.0)

    # strike spacing
    dk = 2.0 * np.pi / (N * eta)
    j  = np.arange(N)
    k  = k0 + j * dk

    # Common complex FFT kernel
    kern = np.exp(-1j * u * k0) * w / np.pi  # 1/pi factor here

    # FFTs
    P  = np.fft.fft(F_price * kern)
    Df = np.fft.fft(F_delta * kern)
    Gf = np.fft.fft(F_gamma * kern)
    Vf = np.fft.fft(F_vega  * kern)

    # Undo damping per strike: exp(-alpha * k)
    damp = np.exp(-alpha * k)

    Price = np.real(damp * P)
    Delta = np.real(damp * Df)
    Gamma = np.real(damp * Gf)
    Vega  = np.real(damp * Vf)

    return k, Price, Delta, Gamma, Vega

def price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                         alpha=1.5, N=2**12, eta=0.25, k0=None):
    """
    Interpolate FFT-based Price/Greeks to a specific strike K.
    """
    k_grid, P, D, G, V = cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                                         alpha, N, eta, k0)
    k = np.log(K)
    # linear interpolation
    if k <= k_grid[0]:
        return float(P[0]), float(D[0]), float(G[0]), float(V[0])
    if k >= k_grid[-1]:
        return float(P[-1]), float(D[-1]), float(G[-1]), float(V[-1])
    idx = np.searchsorted(k_grid, k) - 1
    t = (k - k_grid[idx]) / (k_grid[idx+1] - k_grid[idx])
    price = (1-t)*P[idx] + t*P[idx+1]
    delta = (1-t)*D[idx] + t*D[idx+1]
    gamma = (1-t)*G[idx] + t*G[idx+1]
    vega  = (1-t)*V[idx] + t*V[idx+1]
    return float(price), float(delta), float(gamma), float(vega)

# ==============================
# 4) Demo / Comparison printout
# ==============================
if __name__ == "__main__":
    # Parameters
    S0    = 100.0
    r, q  = 0.00, 0.00
    kappa = 2.0
    theta = 0.04
    sigma = 0.6
    rho   = -0.7
    v0    = 0.05

    strikes    = [90.0, 100.0, 110.0]
    maturities = [0.25, 1.0]

    # Reference quadrature controls
    alpha_ref = 1.5
    N_ref     = 2**14
    u_max_ref = 200.0

    # FFT grid controls
    alpha_fft = 1.5
    N_fft     = 2**12
    eta_fft   = 0.25
    k0_fft    = np.log(S0) - 3.0  # keep fixed (don’t depend on complex S0)

    print("=== Heston: FFT-based vs Reference (Analytic-Integral Greeks) ===\n")
    header = ("T    K     "
              "Price_ref    Price_FFT     "
              "Delta_ref    Delta_FFT     "
              "Gamma_ref     Gamma_FFT     "
              "Vega_ref      Vega_FFT     "
              "|e_P|       |e_D|       |e_G|        |e_V|")
    print(header)
    print("-"*len(header))

    for T in maturities:
        for K in strikes:
            # Reference (single-strike quadrature)
            C_ref = cm_price_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                     alpha=alpha_ref, N=N_ref, u_max=u_max_ref)
            D_ref, G_ref, V_ref = cm_greeks_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                                    alpha=alpha_ref, N=N_ref, u_max=u_max_ref)

            # FFT-based (interpolated at K)
            C_fft, D_fft, G_fft, V_fft = price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                                              alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft)

            eP = abs(C_fft - C_ref)
            eD = abs(D_fft - D_ref)
            eG = abs(G_fft - G_ref)
            eV = abs(V_fft - V_ref)

            print(f"{T:<4.2f} {K:<6.1f} "
                  f"{C_ref:>11.6f}  {C_fft:>11.6f}   "
                  f"{D_ref:>10.6f}  {D_fft:>10.6f}   "
                  f"{G_ref:>11.6e}  {G_fft:>11.6e}   "
                  f"{V_ref:>11.6f}  {V_fft:>11.6f}   "
                  f"{eP:>9.2e}  {eD:>9.2e}  {eG:>9.2e}  {eV:>9.2e}")

    print("\n=== Explicit FFT-based Greeks ===")
    for T in maturities:
        for K in strikes:
            _, D_fft, G_fft, V_fft = price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                                          alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft)
            print(f"T={T:.2f}, K={K:.1f} | FFT-based Delta={D_fft:.6f}, "
                  f"Gamma={G_fft:.6e}, Vega={V_fft:.6f}")


def finite_diff_greeks_from_price(price_fn, S0, K, T, r, q, v0, kappa, theta, sigma, rho):
    # clean finite-difference checks (reference price function ONLY)
    epsS = 1e-4 * S0
    C0   = price_fn(S0, K, T, r, q, v0, kappa, theta, sigma, rho)
    Cp   = price_fn(S0 + epsS, K, T, r, q, v0, kappa, theta, sigma, rho)
    Cm   = price_fn(S0 - epsS, K, T, r, q, v0, kappa, theta, sigma, rho)
    Delta_fd = (Cp - Cm) / (2*epsS)
    Gamma_fd = (Cp - 2*C0 + Cm) / (epsS**2)

    epsv = 1e-6  # bump v0 (absolute)
    Cv_p = price_fn(S0, K, T, r, q, v0 + epsv, kappa, theta, sigma, rho)
    Cv_m = price_fn(S0, K, T, r, q, v0 - epsv, kappa, theta, sigma, rho)
    Vega_fd = (Cv_p - Cv_m) / (2*epsv)
    return Delta_fd, Gamma_fd, Vega_fd

def bs_limit_params(sigma_bs):
    return dict(v0=sigma_bs**2, theta=sigma_bs**2, kappa=1e6, sigma=1e-4, rho=0.0)

def run_checks():
    S0, r, q = 100.0, 0.0, 0.0
    v0, kappa, theta, sigma, rho = 0.05, 2.0, 0.04, 0.6, -0.7
    Ks = [90.0, 100.0, 110.0]; Ts = [0.25, 1.0]

    # tighten reference a bit for testing
    pref = dict(alpha=1.5, N=2**15, u_max=250.0)  # reference quadrature
    pfft = dict(alpha=1.5, N=2**13, eta=0.20, k0=np.log(S0)-3.5)  # fft grid

    print("\n[1] Ref vs FFT convergence sanity")
    for T in Ts:
        for K in Ks:
            C_ref = cm_price_singleK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pref)
            D_ref, G_ref, V_ref = cm_greeks_singleK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pref)
            C_fft, D_fft, G_fft, V_fft = price_greeks_fft_atK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pfft)
            print(f"T={T:.2f},K={K:.1f} | dP={abs(C_fft-C_ref):.2e}, dD={abs(D_fft-D_ref):.2e}, dG={abs(G_fft-G_ref):.2e}, dV={abs(V_fft-V_ref):.2e}")

    print("\n[2] Financial sanity (bounds/monotonicity/convexity)")
    T=0.5
    # Check bounds and monotonicity across K
    grid = np.linspace(70, 130, 13)
    last_price = None
    last_delta = None
    convex_ok = True
    prices = []
    for K in grid:
        C = cm_price_singleK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pref)
        D,G,V = cm_greeks_singleK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pref)
        prices.append(C)
        assert 0.0 <= D <= 1.0 + 1e-6, f"Delta out of bounds: {D}"
        assert G >= -1e-8, f"Gamma negative: {G}"
        assert V >= -1e-6, f"Vega negative: {V}"
        if last_price is not None:
            assert C <= last_price + 1e-8, "Price did not decrease with K"
            assert D <= last_delta + 1e-6, "Delta did not decrease with K"
        last_price, last_delta = C, D
    # Convexity via discrete second difference
    for i in range(1, len(grid)-1):
        if prices[i-1] - 2*prices[i] + prices[i+1] < -1e-4:
            convex_ok = False
    print("Convexity OK:", convex_ok)

    print("\n[3] Finite-difference vs analytic-integrand Greeks (reference)")
    for T in Ts:
        for K in Ks:
            D_ref, G_ref, V_ref = cm_greeks_singleK(S0,K,T,r,q,v0,kappa,theta,sigma,rho, **pref)
            D_fd, G_fd, V_fd = finite_diff_greeks_from_price(cm_price_singleK, S0,K,T,r,q,v0,kappa,theta,sigma,rho)
            print(f"T={T:.2f},K={K:.1f} | Δ err={abs(D_ref-D_fd):.2e}, Γ err={abs(G_ref-G_fd):.2e}, V err={abs(V_ref-V_fd):.2e}")

    print("\n[4] Black–Scholes limit (prices+Greeks)")
    sigma_bs = 0.20
    hpar = bs_limit_params(sigma_bs)
    from math import log, sqrt, exp
    from math import erf
    def N(x): return 0.5*(1+erf(x/np.sqrt(2)))
    for T in Ts:
        for K in Ks:
            # BS closed form
            d1 = (np.log(S0/K) + (r - q + 0.5*sigma_bs**2)*T)/(sigma_bs*np.sqrt(T))
            d2 = d1 - sigma_bs*np.sqrt(T)
            C_bs = S0*np.exp(-q*T)*N(d1) - K*np.exp(-r*T)*N(d2)
            Delta_bs = np.exp(-q*T)*N(d1)
            Gamma_bs = np.exp(-q*T)*np.exp(-0.5*d1*d1)/(S0*sigma_bs*np.sqrt(T)*np.sqrt(2*np.pi))
            Vega_bs  = S0*np.exp(-q*T)*np.sqrt(T)*np.exp(-0.5*d1*d1)/np.sqrt(2*np.pi)  # w.r.t. sigma; we compare trends only

            # Heston->BS via limit params; Greeks w.r.t v0 (note: not same as BS vega wrt sigma)
            C_ref = cm_price_singleK(S0, K, T, r, q, **hpar, **hpar, rho=0.0, **dict())  # (we'll just do FFT route for brevity)
            C_fft, D_fft, G_fft, V_fft = price_greeks_fft_atK(S0, K, T, r, q, **hpar, **hpar, rho=0.0)

            print(f"T={T:.2f},K={K:.1f} | BS C={C_bs:.6f}, Heston->BS C_FFT={C_fft:.6f}, Δ_BS={Delta_bs:.6f}, Δ_FFT={D_fft:.6f}, Γ_BS≈{Gamma_bs:.3e}, Γ_FFT={G_fft:.3e}")



=== Heston: FFT-based vs Reference (Analytic-Integral Greeks) ===

T    K     Price_ref    Price_FFT     Delta_ref    Delta_FFT     Gamma_ref     Gamma_FFT     Vega_ref      Vega_FFT     |e_P|       |e_D|       |e_G|        |e_V|
------------------------------------------------------------------------------------------------------------------------------------------------------------------
0.25 90.0     11.281628    11.281855     0.867935    0.867886   1.420199e-02  1.420653e-02     20.864669    20.866928    2.27e-04   4.96e-05   4.54e-06   2.26e-03
0.25 100.0     4.074919     4.075344     0.607988    0.607946   3.843484e-02  3.843594e-02     36.982640    36.978138    4.25e-04   4.20e-05   1.10e-06   4.50e-03
0.25 110.0     0.559669     0.561398     0.163908    0.164172   3.869296e-02  3.866444e-02     20.441050    20.446450    1.73e-03   2.65e-04   2.85e-05   5.40e-03
1.00 90.0     13.993240    13.993403     0.811730    0.811704   1.145055e-02  1.145220e-02     30.949678    30.949529 

The above table is reported in the work at the relevant tables.

---



## Black-Scholes limit verification for prices and greesk via Heston-FFT

In [ ]:

import numpy as np
from math import log, sqrt, exp, erf, pi

# -----------------------------
# Black–Scholes closed-form (with ∂C/∂v0)
# -----------------------------
def _N(x):
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def _phi(x):
    return (1.0 / sqrt(2.0 * pi)) * np.exp(-0.5 * x * x)

def bs_price_delta_gamma_vega_v0(S0, K, T, r, q, sigma_bs):
    """
    Return (Price, Delta, Gamma, Vega_v0) in Black–Scholes.
    Vega_v0 = ∂C/∂v0 with v0 = sigma^2, so Vega_v0 = (∂C/∂σ)/(2σ).
    """
    if T <= 0:
        payoff = max(S0 - K, 0.0)
        return payoff, float(S0 > K), 0.0, 0.0

    sig = sigma_bs
    d1 = (log(S0/K) + (r - q + 0.5*sig*sig)*T) / (sig * sqrt(T))
    d2 = d1 - sig*sqrt(T)

    disc_r = exp(-r*T)
    disc_q = exp(-q*T)

    price = S0*disc_q*_N(d1) - K*disc_r*_N(d2)
    delta = disc_q*_N(d1)
    gamma = disc_q*_phi(d1) / (S0 * sig * sqrt(T))
    vega_sigma = S0 * disc_q * sqrt(T) * _phi(d1)  # ∂C/∂σ
    vega_v0 = vega_sigma / (2.0 * sig)            # ∂C/∂v0 with v0 = σ^2
    return float(price), float(delta), float(gamma), float(vega_v0)

# -----------------------------
# Heston CF (Lord–Kahl “little trap”) — stable branch
# -----------------------------
def heston_charfunc_lord_kahl(u, T, v0, kappa, theta, sigma, rho, r, q, return_CD=False):
    """
    Stable Heston CF:
      phi(u) = exp(C(u,T) + D(u,T) * v0)
    Returns (phi[, C, D]).
    """
    iu = 1j * u
    a  = kappa * theta
    b  = kappa - rho * sigma * iu
    d  = np.sqrt(b*b + (iu + u*u) * sigma * sigma)
    g  = (b - d) / (b + d)
    e  = np.exp(-d * T)
    C  = (r - q) * iu * T + (a / (sigma*sigma)) * ((b - d) * T - 2.0 * np.log((1 - g*e) / (1 - g)))
    D  = ((b - d) / (sigma*sigma)) * ((1 - e) / (1 - g*e))
    phi = np.exp(C + D * v0)
    if return_CD:
        return phi, C, D
    return phi

# -----------------------------
# Carr–Madan FFT: Prices & Greeks on a strike grid
# (trapezoidal weights; alpha-damped; analytic-integrand Greeks)
# -----------------------------
def cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                    alpha=1.5, N=2**12, eta=0.25, k0=None):
    """
    Return (k_grid, Price, Delta, Gamma, Vega_v0) arrays (length N).
    - Trapezoidal weights (compatible with power-of-two N)
    - Analytic-integrand Greeks:
        Delta: multiply by (i z / S0)
        Gamma: multiply by (-(i z + z^2) / S0^2)
        Vega_v0: multiply by D(z) (since phi = exp(C + D v0))
    - Strike spacing: dk = 2π/(N*eta) with phase centering at k0
    """
    if T <= 0:
        raise ValueError("Use T>0 for transform-based pricing.")

    # Frequency grid
    m = np.arange(N, dtype=float)
    u = m * eta
    beta = alpha + 1.0
    z = u - 1j * beta

    # Affine CF at shifted arg
    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    S_term = np.exp(1j * z * np.log(S0))  # S0^{i z}

    # Carr–Madan denominator
    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u

    # Common base
    base = np.exp(-r*T) * (S_term * phi) / denom

    # Frequency-domain inputs
    F_price = base
    F_delta = base * (1j * z / S0)
    F_gamma = base * (-(1j * z + z*z) / (S0*S0))
    F_vega  = base * D

    # Trapezoidal weights in frequency
    w = np.ones(N); w[0] = 0.5; w[-1] = 0.5
    w *= eta

    # Strike grid mapping
    dk = 2.0 * np.pi / (N * eta)
    if k0 is None:
        # center the window around log(S0)
        k0 = np.log(S0) - 0.5 * N * dk
    j  = np.arange(N)
    k  = k0 + j * dk  # log-strike grid

    # Kernel: phase shift to k0 and 1/π factor
    kern = np.exp(-1j * u * k0) * w / np.pi

    # FFTs
    P  = np.fft.fft(F_price * kern)
    Df = np.fft.fft(F_delta * kern)
    Gf = np.fft.fft(F_gamma * kern)
    Vf = np.fft.fft(F_vega  * kern)

    # Remove damping in strike domain
    damp = np.exp(-alpha * k)
    Price = np.real(damp * P)
    Delta = np.real(damp * Df)
    Gamma = np.real(damp * Gf)
    Vega  = np.real(damp * Vf)

    return k, Price, Delta, Gamma, Vega

def price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                         alpha=1.5, N=2**12, eta=0.25, k0=None):
    """
    Interpolate FFT-based (Price, Delta, Gamma, Vega_v0) to a specific strike K.
    Raises if K is outside the FFT grid (avoid hidden extrapolation).
    """
    k_grid, P, D, G, V = cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                                         alpha=alpha, N=N, eta=eta, k0=k0)
    k = np.log(K)
    if not (k_grid[0] <= k <= k_grid[-1]):
        raise ValueError("K outside FFT k-grid; enlarge N or adjust eta/k0")
    idx = np.searchsorted(k_grid, k) - 1
    t = (k - k_grid[idx]) / (k_grid[idx+1] - k_grid[idx])
    price = (1-t)*P[idx] + t*P[idx+1]
    delta = (1-t)*D[idx] + t*D[idx+1]
    gamma = (1-t)*G[idx] + t*G[idx+1]
    vega  = (1-t)*V[idx] + t*V[idx+1]
    return float(price), float(delta), float(gamma), float(vega)

# -----------------------------
# Near–Black–Scholes Heston params (numerically gentle)
# -----------------------------
def heston_params_bs_limit(sigma_bs):
    """
    Choose Heston params so variance is (almost) deterministic at σ_BS^2:
      v0 = theta = σ_BS^2,  rho = 0,  kappa moderately large,  vol-of-vol σ very small.
    Extremely large kappa / tiny sigma cause stiffness; avoid that here.
    """
    return {
        "v0":    sigma_bs**2,
        "theta": sigma_bs**2,
        "kappa": 100.0,   # not 1e6
        "sigma": 1e-3,    # not 1e-4
        "rho":   0.0
    }

# -----------------------------
# Driver: verify BS limit across a small grid
# -----------------------------
def verify_bs_limit_across_grid(
    S0=100.0, r=0.0, q=0.0,
    sigma_bs=0.20,
    Ks=(90.0, 100.0, 110.0),
    Ts=(0.25, 1.00),
    alpha_fft=1.5, N_fft=2**12, eta_fft=0.20, k0_fft=None,
    gates = dict(max_eP=5e-4, max_eD=1e-4, max_eG=5e-5, max_eV=5e-3)
):
    if k0_fft is None:
        # center window around log S0
        dk = 2.0*np.pi/(N_fft*eta_fft)
        k0_fft = np.log(S0) - 0.5 * N_fft * dk

    hp = heston_params_bs_limit(sigma_bs)

    print("=== Black–Scholes Limit Verification (BS closed-form vs Heston-FFT in BS-limit) ===")
    print("Note: Vega reported is ∂C/∂v0; BS vega is converted via ∂C/∂v0 = (∂C/∂σ)/(2σ).")
    print(f"sigma_BS={sigma_bs:.4f}, v0=theta={sigma_bs**2:.6f}, "
          f"kappa={hp['kappa']:.1f}, sigma(vol-of-vol)={hp['sigma']:.1e}, rho={hp['rho']:.1f}")
    print()
    print("T     K      Price_BS    Price_FFT    |e_P|      Delta_BS   Delta_FFT   |e_D|      "
          "Gamma_BS    Gamma_FFT   |e_G|      Vega_v0_BS  Vega_v0_FFT  |e_V|")
    print("-"*168)

    max_eP = max_eD = max_eG = max_eV = 0.0

    for T in Ts:
        for K in Ks:
            # BS closed form
            P_bs, D_bs, G_bs, Vv0_bs = bs_price_delta_gamma_vega_v0(S0, K, T, r, q, sigma_bs)
            # Heston-FFT in BS-limit
            P_fft, D_fft, G_fft, Vv0_fft = price_greeks_fft_atK(
                S0, K, T, r, q,
                hp["v0"], hp["kappa"], hp["theta"], hp["sigma"], hp["rho"],
                alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft
            )
            # Errors
            eP = abs(P_fft - P_bs); max_eP = max(max_eP, eP)
            eD = abs(D_fft - D_bs); max_eD = max(max_eD, eD)
            eG = abs(G_fft - G_bs); max_eG = max(max_eG, eG)
            eV = abs(Vv0_fft - Vv0_bs); max_eV = max(max_eV, eV)

            print(f"{T:0.2f}  {K:6.1f}  {P_bs:11.6f}  {P_fft:11.6f}  {eP:9.2e}   "
                  f"{D_bs:10.6f}  {D_fft:10.6f}  {eD:9.2e}   "
                  f"{G_bs:10.6e}  {G_fft:10.6e}  {eG:9.2e}   "
                  f"{Vv0_bs:11.6f}  {Vv0_fft:11.6f}  {eV:9.2e}")

    print("\nSuggested acceptance gates (near-BS limit): "
          f"max|e_P|<{gates['max_eP']:.1e}, max|e_D|<{gates['max_eD']:.1e}, "
          f"max|e_G|<{gates['max_eG']:.1e}, max|e_V|<{gates['max_eV']:.1e}")
    print(f"Observed maxima: max|e_P|={max_eP:.2e}, max|e_D|={max_eD:.2e}, "
          f"max|e_G|={max_eG:.2e}, max|e_V|={max_eV:.2e}")

# -----------------------------
# Run the check
# -----------------------------
if __name__ == "__main__":
    verify_bs_limit_across_grid(
        S0=100.0, r=0.0, q=0.0,
        sigma_bs=0.20,
        Ks=(90.0, 100.0, 110.0),
        Ts=(0.25, 1.00),
        alpha_fft=1.5,
        N_fft=2**13,         # a bit finer than 2**12 for tighter match
        eta_fft=0.20,        # slightly smaller eta to push aliasing further out
        k0_fft=None          # auto-center around log S0
    )


=== Black–Scholes Limit Verification (BS closed-form vs Heston-FFT in BS-limit) ===
Note: Vega reported is ∂C/∂v0; BS vega is converted via ∂C/∂v0 = (∂C/∂σ)/(2σ).
sigma_BS=0.2000, v0=theta=0.040000, kappa=100.0, sigma(vol-of-vol)=1.0e-03, rho=0.0

T     K      Price_BS    Price_FFT    |e_P|      Delta_BS   Delta_FFT   |e_D|      Gamma_BS    Gamma_FFT   |e_G|      Vega_v0_BS  Vega_v0_FFT  |e_V|
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0.25    90.0    10.712381    10.712639   2.59e-04     0.865118    0.865074   4.39e-05   2.169885e-02  2.169973e-02   8.72e-07     27.123568     1.084986   2.60e+01
0.25   100.0     3.987761     3.987761   9.06e-09     0.519939    0.519939   1.05e-09   3.984439e-02  3.984439e-02   8.44e-11     49.805489     1.992220   4.78e+01
0.25   110.0     0.953947     0.954178   2.30e-04     0.183236    0.183258   2.20e-05   2.653422e-02  2.65



---



#Part 2

## Visualizations

In [ ]:
# ============================================
# 2×3 matrix figures per Greek + 2×2 slices
# ============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


S0    = 100.0
r, q  = 0.0, 0.0
kappa = 2.0
theta = 0.04
sigma = 0.6
rho   = -0.7
v0    = 0.05

# Reference (ground-truth) quadrature
alpha_ref = 1.5
N_ref     = 2**15
u_max_ref = 250.0

# FFT baseline
alpha_fft = 1.5
N_fft     = 2**12
eta_fft   = 0.25
k0_fft    = np.log(S0) - 3.0

# Grids
Ks = np.linspace(80, 120, 9)
Ts = np.array([0.25, 0.5, 1.0])

# ATM point for convergence/sensitivity
K_ATM, T_ATM = 100.0, 0.25

# ---------- Helpers ----------
def _error_metrics(ref, approx, eps=1e-12):
    abs_err = np.abs(approx - ref)
    rel_err = abs_err / (np.abs(ref) + eps)
    return abs_err, rel_err

def _compute_grid(S0, r, q, v0, kappa, theta, sigma, rho,
                  Ks, Ts, alpha_ref, N_ref, u_max_ref,
                  alpha_fft, N_fft, eta_fft, k0_fft):
    rows = []
    for T in Ts:
        for K in Ks:
            C_ref = cm_price_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                     alpha=alpha_ref, N=N_ref, u_max=u_max_ref)
            D_ref, G_ref, V_ref = cm_greeks_singleK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                                    alpha=alpha_ref, N=N_ref, u_max=u_max_ref)
            C_fft, D_fft, G_fft, V_fft = price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                                              alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft)
            rows.append(dict(T=T, K=K,
                             P_ref=C_ref, P_fft=C_fft,
                             D_ref=D_ref, D_fft=D_fft,
                             G_ref=G_ref, G_fft=G_fft,
                             V_ref=V_ref, V_fft=V_fft))
    return pd.DataFrame(rows).sort_values(["T","K"]).reset_index(drop=True)

def _gridify(df, col, Ks, Ts):
    M = np.zeros((len(Ts), len(Ks)))
    for i, T in enumerate(Ts):
        M[i,:] = df[df["T"]==T].sort_values("K")[col].to_numpy()
    return M

# Map names
_LABELS = {
    "price": ("P_ref","P_fft","Price"),
    "delta": ("D_ref","D_fft","Delta"),
    "gamma": ("G_ref","G_fft","Gamma"),
    "vega" : ("V_ref","V_fft","Vega"),
}

# ---------- Core: build 2×3 matrix & 2×2 slices ----------
def greek_matrix_and_slices(metric: str,
                            outdir="figs_matrix",
                            Ns=(2**11,2**12,2**13,2**14),
                            etas=(0.20,0.25,0.30),
                            k0_shifts=(-4.0,-3.5,-3.0,-2.5,-2.0),
                            alphas=(1.0,1.25,1.5,1.75,2.0),
                            K_slices=(90.0, 100.0, 110.0),
                            T_slices=(0.25, 0.5, 1.0)):
    """
    metric in {'price','delta','gamma','vega'}.
    Produces:
      - 2×3 matrix figure: heatmap | errors vs N | errors vs eta
                            errors vs k0 | errors vs alpha | histogram(rel)
      - 2×2 slices figure: values vs K + abs error vs K; values vs T + abs error vs T
    Saves PNGs and returns their paths.
    """
    os.makedirs(outdir, exist_ok=True)
    ref_col, fft_col, pretty = _LABELS[metric]

    # Build grid (ref & fft)
    df = _compute_grid(S0, r, q, v0, kappa, theta, sigma, rho,
                       Ks, Ts, alpha_ref, N_ref, u_max_ref,
                       alpha_fft, N_fft, eta_fft, k0_fft)
    df["e_abs_"+metric] = np.abs(df[fft_col] - df[ref_col])
    df["e_rel_"+metric] = df["e_abs_"+metric] / (np.abs(df[ref_col]) + 1e-12)

    # ---------- (1) 2×3 MATRIX ----------
    fig, axes = plt.subplots(2, 3, figsize=(11.0, 7.0))

    # A. Heatmap of abs error
    M = _gridify(df, "e_abs_"+metric, Ks, Ts)
    ax = axes[0,0]
    im = ax.imshow(M, aspect='auto', origin='lower',
                   extent=(Ks[0], Ks[-1], Ts[0], Ts[-1]))
    fig.colorbar(im, ax=ax)
    ax.set_title(f"{pretty} abs error")
    ax.set_xlabel("Strike K"); ax.set_ylabel("T")

    # Gather ATM refs once
    Pref  = cm_price_singleK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                             alpha=alpha_ref, N=N_ref, u_max=u_max_ref)
    Dref, Gref, Vref = cm_greeks_singleK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                                         alpha=alpha_ref, N=N_ref, u_max=u_max_ref)
    REF = {"price": Pref, "delta": Dref, "gamma": Gref, "vega": Vref}[metric]

    # B. Errors vs N (log–log) at ATM
    eN = []
    for N in Ns:
        C, D, G, V = price_greeks_fft_atK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                                          alpha=alpha_fft, N=N, eta=eta_fft, k0=k0_fft)
        FFT = {"price": C, "delta": D, "gamma": G, "vega": V}[metric]
        eN.append(abs(FFT - REF))
    ax = axes[0,1]
    ax.loglog(Ns, eN, marker='o')
    ax.set_title(f"{pretty}: error vs N (ATM)")
    ax.set_xlabel("N"); ax.set_ylabel("abs error")

    # C. Errors vs eta at ATM
    eEta = []
    for et in etas:
        C, D, G, V = price_greeks_fft_atK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                                          alpha=alpha_fft, N=N_fft, eta=et, k0=k0_fft)
        FFT = {"price": C, "delta": D, "gamma": G, "vega": V}[metric]
        eEta.append(abs(FFT - REF))
    ax = axes[0,2]
    ax.plot(etas, eEta, marker='o')
    ax.set_title(f"{pretty}: error vs η (ATM)")
    ax.set_xlabel("η"); ax.set_ylabel("abs error")

    # D. Sensitivity vs k0 at ATM
    eK0 = []
    for shift in k0_shifts:
        k0h = np.log(S0) + shift
        C, D, G, V = price_greeks_fft_atK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                                          alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0h)
        FFT = {"price": C, "delta": D, "gamma": G, "vega": V}[metric]
        eK0.append(abs(FFT - REF))
    ax = axes[1,0]
    ax.plot(k0_shifts, eK0, marker='o')
    ax.set_title(f"{pretty}: sensitivity to $k_0$ (ATM)")
    ax.set_xlabel("k0 shift"); ax.set_ylabel("abs error")

    # E. Sensitivity vs alpha at ATM
    eA = []
    for a in alphas:
        C, D, G, V = price_greeks_fft_atK(S0, K_ATM, T_ATM, r, q, v0, kappa, theta, sigma, rho,
                                          alpha=a, N=N_fft, eta=eta_fft, k0=k0_fft)
        FFT = {"price": C, "delta": D, "gamma": G, "vega": V}[metric]
        eA.append(abs(FFT - REF))
    ax = axes[1,1]
    ax.plot(alphas, eA, marker='o')
    ax.set_title(f"{pretty}: sensitivity to $\\alpha$ (ATM)")
    ax.set_xlabel("α"); ax.set_ylabel("abs error")

    # F. Histogram of relative errors over (K,T)
    ax = axes[1,2]
    ax.hist(df["e_rel_"+metric].to_numpy(), bins=20)
    ax.set_title(f"{pretty}: relative error over $(K,T)$")
    ax.set_xlabel("relative error"); ax.set_ylabel("count")

    fig.tight_layout()
    matrix_path = os.path.join(outdir, f"{metric}_matrix_2x3.png")
    fig.savefig(matrix_path, dpi=300)
    plt.close(fig)

    # ---------- (2) 2×2 SLICES ----------
    fig2, axes2 = plt.subplots(2, 2, figsize=(10.0, 7.0))

    # Values vs K at fixed T (show ref & FFT distinctly)
    Tfix = T_slices[0]
    subK = df[df["T"]==Tfix].sort_values("K")
    ax = axes2[0,0]
    # Distinct linestyles/markers to ensure both are visible even if overlaid
    ax.plot(subK["K"], subK[ref_col], marker='o', linestyle='-', label=f"{pretty} ref")
    ax.plot(subK["K"], subK[fft_col], marker='x', linestyle='--', label=f"{pretty} FFT")
    ax.set_title(f"{pretty} vs K at T={Tfix}")
    ax.set_xlabel("K"); ax.set_ylabel(pretty); ax.legend()

    # Error vs K at same T
    ax = axes2[0,1]
    ax.plot(subK["K"], np.abs(subK[fft_col]-subK[ref_col]), marker='o')
    ax.set_title(f"{pretty} abs error vs K (T={Tfix})")
    ax.set_xlabel("K"); ax.set_ylabel("abs error")

    # Values vs T at fixed K (show ref & FFT distinctly)
    Kfix = K_slices[1]
    subT = df[df["K"]==Kfix].sort_values("T")
    ax = axes2[1,0]
    ax.plot(subT["T"], subT[ref_col], marker='o', linestyle='-', label=f"{pretty} ref")
    ax.plot(subT["T"], subT[fft_col], marker='x', linestyle='--', label=f"{pretty} FFT")
    ax.set_title(f"{pretty} vs T at K={Kfix}")
    ax.set_xlabel("T"); ax.set_ylabel(pretty); ax.legend()

    # Error vs T at same K
    ax = axes2[1,1]
    ax.plot(subT["T"], np.abs(subT[fft_col]-subT[ref_col]), marker='o')
    ax.set_title(f"{pretty} abs error vs T (K={Kfix})")
    ax.set_xlabel("T"); ax.set_ylabel("abs error")

    fig2.tight_layout()
    slices_path = os.path.join(outdir, f"{metric}_slices_2x2.png")
    fig2.savefig(slices_path, dpi=300)
    plt.close(fig2)



    return matrix_path, slices_path

# ---------- Generate for Delta/Gamma/Vega ----------
delta_paths = greek_matrix_and_slices("delta", outdir="figs_matrix")
gamma_paths = greek_matrix_and_slices("gamma", outdir="figs_matrix")
vega_paths  = greek_matrix_and_slices("vega",  outdir="figs_matrix")

print("Saved Delta figures/snippet:", delta_paths)
print("Saved Gamma figures/snippet:", gamma_paths)
print("Saved Vega  figures/snippet:", vega_paths)


Saved Delta figures/snippet: ('figs_matrix/delta_matrix_2x3.png', 'figs_matrix/delta_slices_2x2.png')
Saved Gamma figures/snippet: ('figs_matrix/gamma_matrix_2x3.png', 'figs_matrix/gamma_slices_2x2.png')
Saved Vega  figures/snippet: ('figs_matrix/vega_matrix_2x3.png', 'figs_matrix/vega_slices_2x2.png')




---

# Part 3

Code for benchmarking of Price and Greeks of Quadrature based versus FFT

*   Price, $\Delta, \Gamma, Vega_{v_0} $ tables: Reference (quadrature) vs FFT
*   Heston in a near–Black–Scholes regime



---




In [ ]:


import numpy as np
from math import log, sqrt, exp, erf, pi

# ---------- Normal helpers + BS closed-form (optional external check) ----------
def _N(x):  # CDF
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def _phi(x):  # PDF
    return (1.0 / sqrt(2.0 * pi)) * np.exp(-0.5 * x * x)

def bs_price_delta_gamma_vega_v0(S0, K, T, r, q, sigma_bs):
    if T <= 0:
        payoff = max(S0 - K, 0.0)
        return payoff, float(S0 > K), 0.0, 0.0
    sig = sigma_bs
    d1 = (log(S0/K) + (r - q + 0.5*sig*sig)*T) / (sig * sqrt(T))
    d2 = d1 - sig*sqrt(T)
    disc_r = exp(-r*T); disc_q = exp(-q*T)
    price = S0*disc_q*_N(d1) - K*disc_r*_N(d2)
    delta = disc_q*_N(d1)
    gamma = disc_q*_phi(d1) / (S0 * sig * sqrt(T))
    vega_sigma = S0 * disc_q * sqrt(T) * _phi(d1)  # ∂C/∂σ
    vega_v0 = vega_sigma / (2.0 * sig)            # ∂C/∂v0 with v0 = σ^2
    return float(price), float(delta), float(gamma), float(vega_v0)

# ---------- Heston CF (Lord–Kahl “little trap”) ----------
def heston_charfunc_lord_kahl(u, T, v0, kappa, theta, sigma, rho, r, q, return_CD=False):
    iu = 1j * u
    a  = kappa * theta
    b  = kappa - rho * sigma * iu
    d  = np.sqrt(b*b + (iu + u*u) * sigma * sigma)
    g  = (b - d) / (b + d)
    e  = np.exp(-d * T)
    C  = (r - q) * iu * T + (a / (sigma*sigma)) * ((b - d) * T - 2.0 * np.log((1 - g*e) / (1 - g)))
    D  = ((b - d) / (sigma*sigma)) * ((1 - e) / (1 - g*e))
    phi = np.exp(C + D * v0)
    if return_CD:
        return phi, C, D
    return phi

# ---------- Carr–Madan FFT to strike grid (TRAPEZOIDAL weights) ----------
def cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                    alpha=1.5, N=2**12, eta=0.25, k0=None):
    if T <= 0:
        raise ValueError("Use T>0 for transform-based pricing.")

    m = np.arange(N, dtype=float)
    u = m * eta
    beta = alpha + 1.0
    z = u - 1j * beta

    # CF at shifted argument and spot factor
    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    S_term = np.exp(1j * z * np.log(S0))

    # Denominator and base
    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    base  = np.exp(-r*T) * (S_term * phi) / denom

    # Frequency-space inputs (analytic-integrand Greeks)
    F_price = base
    F_delta = base * (1j * z / S0)
    F_gamma = base * (-(1j * z + z*z) / (S0*S0))
    F_vega  = base * D

    # TRAPEZOIDAL weights (compatible with N=2^m)
    w = np.ones(N)
        if N >= 3:
            w[1:-1:2] = 4.0
            w[2:-1:2] = 2.0
    w *= (eta/3.0)

    # Strike grid mapping and centering
    dk = 2.0 * np.pi / (N * eta)
    if k0 is None:
        k0 = np.log(S0) - 0.5 * N * dk
    j  = np.arange(N)
    k  = k0 + j * dk  # log-strike grid

    # Kernel (phase to k0) and 1/π factor
    kern = np.exp(-1j * u * k0) * w / np.pi

    # FFTs
    P  = np.fft.fft(F_price * kern)
    Df = np.fft.fft(F_delta * kern)
    Gf = np.fft.fft(F_gamma * kern)
    Vf = np.fft.fft(F_vega  * kern)

    # Remove damping in strike domain
    damp  = np.exp(-alpha * k)
    Price = np.real(damp * P)
    Delta = np.real(damp * Df)
    Gamma = np.real(damp * Gf)
    Vega  = np.real(damp * Vf)

    return k, Price, Delta, Gamma, Vega

def price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                         alpha=1.5, N=2**12, eta=0.25, k0=None):
    k_grid, P, D, G, V = cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                                         alpha=alpha, N=N, eta=eta, k0=k0)
    k = np.log(K)
    if not (k_grid[0] <= k <= k_grid[-1]):
        raise ValueError("K outside FFT k-grid; enlarge N or adjust eta/k0")
    idx = np.searchsorted(k_grid, k) - 1
    t = (k - k_grid[idx]) / (k_grid[idx+1] - k_grid[idx])
    price = (1-t)*P[idx] + t*P[idx+1]
    delta = (1-t)*D[idx] + t*D[idx+1]
    gamma = (1-t)*G[idx] + t*G[idx+1]
    vega  = (1-t)*V[idx] + t*V[idx+1]
    return float(price), float(delta), float(gamma), float(vega)

# ---------- Single-strike REFERENCE quadrature (Simpson, correct parity) ----------
def price_delta_gamma_vega_quadrature_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                                          alpha=1.5, umax=200.0, Nq=2**15):
    """
    Use composite Simpson on [0,umax]. Requires Nq odd (Nq-1 even subintervals).
    If Nq is even, we fall back to trapezoid for robustness.
    """
    u  = np.linspace(0.0, umax, Nq)
    du = u[1] - u[0]
    beta = alpha + 1.0
    z = u - 1j * beta
    k = np.log(K)

    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    S_term = np.exp(1j * z * np.log(S0))
    denom  = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    base   = np.exp(-r*T) * (S_term * phi) / denom
    phase  = np.exp(-1j * u * k)

    Gp = base * phase
    Gd = base * (1j * z / S0) * phase
    Gg = base * (-(1j * z + z*z) / (S0*S0)) * phase
    Gv = base * D * phase

    if (Nq - 1) % 2 == 0:
        # Simpson
        w = np.ones(Nq); w[1:-1:2] = 4.0; w[2:-1:2] = 2.0
        W = (du / 3.0) * w
    else:
        # Trapezoid fallback
        w = np.ones(Nq); w[0] = 0.5; w[-1] = 0.5
        W = du * w

    pref = np.exp(-alpha * k) / np.pi
    P    = np.real(pref * np.sum(Gp * W))
    Dlt  = np.real(pref * np.sum(Gd * W))
    Gmm  = np.real(pref * np.sum(Gg * W))
    Vv0  = np.real(pref * np.sum(Gv * W))
    return float(P), float(Dlt), float(Gmm), float(Vv0)

# ---------- Near–Black–Scholes Heston parameters (numerically gentle) ----------
def heston_params_bs_limit(sigma_bs):
    return {"v0": sigma_bs**2, "theta": sigma_bs**2, "kappa": 100.0, "sigma": 1e-3, "rho": 0.0}

# ---------- Pretty printers ----------
def table_delta(S0, r, q, sigma_bs, Ks, Ts, alpha_q, umax, Nq, alpha_fft, N_fft, eta_fft, k0_fft):
    hp = heston_params_bs_limit(sigma_bs)
    print("Table: Delta comparison (Reference vs FFT) in near–BS Heston")
    print(f"Ref: Nq={Nq}, umax={umax}, alpha={alpha_q}.  FFT: N={N_fft}, eta={eta_fft}, alpha={alpha_fft}.")
    print()
    print(" T     K      Price_ref    Price_FFT     |e_P|        Delta_ref    Delta_FFT     |e_Δ|")
    print("-"*100)
    for T in Ts:
        for K in Ks:
            Pref, Dref, _, _ = price_delta_gamma_vega_quadrature_atK(
                S0, K, T, r, q, **hp, alpha=alpha_q, umax=umax, Nq=Nq
            )
            Pfft, Dfft, _, _ = price_greeks_fft_atK(
                S0, K, T, r, q, **hp, alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft
            )
            print(f"{T:0.2f}  {K:6.1f}  {Pref:11.6f}  {Pfft:11.6f}  {abs(Pfft-Pref):9.2e}   "
                  f"{Dref:12.6f}  {Dfft:12.6f}  {abs(Dfft-Dref):9.2e}")

def table_gamma(S0, r, q, sigma_bs, Ks, Ts, alpha_q, umax, Nq, alpha_fft, N_fft, eta_fft, k0_fft):
    hp = heston_params_bs_limit(sigma_bs)
    print("\nTable: Gamma comparison (Reference vs FFT) in near–BS Heston")
    print(f"Ref: Nq={Nq}, umax={umax}, alpha={alpha_q}.  FFT: N={N_fft}, eta={eta_fft}, alpha={alpha_fft}.")
    print()
    print(" T     K      Price_ref    Price_FFT     |e_P|         Gamma_ref     Gamma_FFT     |e_Γ|")
    print("-"*100)
    for T in Ts:
        for K in Ks:
            Pref, _, Gref, _ = price_delta_gamma_vega_quadrature_atK(
                S0, K, T, r, q, **hp, alpha=alpha_q, umax=umax, Nq=Nq
            )
            Pfft, _, Gfft, _ = price_greeks_fft_atK(
                S0, K, T, r, q, **hp, alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft
            )
            print(f"{T:0.2f}  {K:6.1f}  {Pref:11.6f}  {Pfft:11.6f}  {abs(Pfft-Pref):9.2e}   "
                  f"{Gref:12.6e}  {Gfft:12.6e}  {abs(Gfft-Gref):9.2e}")

def table_vega(S0, r, q, sigma_bs, Ks, Ts, alpha_q, umax, Nq, alpha_fft, N_fft, eta_fft, k0_fft):
    hp = heston_params_bs_limit(sigma_bs)
    print("\nTable: Vega_{v0} comparison (Reference vs FFT) in near–BS Heston")
    print(f"Ref: Nq={Nq}, umax={umax}, alpha={alpha_q}.  FFT: N={N_fft}, eta={eta_fft}, alpha={alpha_fft}.")
    print("Vega is w.r.t. v0 (initial variance).")
    print()
    print(" T     K      Price_ref    Price_FFT     |e_P|        Vega_v0_ref   Vega_v0_FFT    |e_V|")
    print("-"*100)
    for T in Ts:
        for K in Ks:
            Pref, _, _, Vref = price_delta_gamma_vega_quadrature_atK(
                S0, K, T, r, q, **hp, alpha=alpha_q, umax=umax, Nq=Nq
            )
            Pfft, _, _, Vfft = price_greeks_fft_atK(
                S0, K, T, r, q, **hp, alpha=alpha_fft, N=N_fft, eta=eta_fft, k0=k0_fft
            )
            print(f"{T:0.2f}  {K:6.1f}  {Pref:11.6f}  {Pfft:11.6f}  {abs(Pfft-Pref):9.2e}   "
                  f"{Vref:12.6f}  {Vfft:12.6f}  {abs(Vfft-Vref):9.2e}")

# ---------- Driver ----------
if __name__ == "__main__":
    S0=100.0; r=0.0; q=0.0; sigma_bs=0.20
    Ks=(90.0, 100.0, 110.0)
    Ts=(0.25, 1.00)

    # Reference quadrature (choose Nq odd for Simpson)
    alpha_q=1.5; umax=250.0; Nq=2**15

    # FFT settings (centered window; slightly finer than 2**12)
    alpha_fft=1.5; N_fft=2**13; eta_fft=0.20
    dk = 2.0*np.pi/(N_fft*eta_fft)
    k0_fft = np.log(S0) - 0.5 * N_fft * dk

    table_delta(S0,r,q,sigma_bs,Ks,Ts,alpha_q,umax,Nq,alpha_fft,N_fft,eta_fft,k0_fft)
    table_gamma(S0,r,q,sigma_bs,Ks,Ts,alpha_q,umax,Nq,alpha_fft,N_fft,eta_fft,k0_fft)
    table_vega (S0,r,q,sigma_bs,Ks,Ts,alpha_q,umax,Nq,alpha_fft,N_fft,eta_fft,k0_fft)


Table: Delta comparison (Reference vs FFT) in near–BS Heston
Ref: Nq=32768, umax=250.0, alpha=1.5.  FFT: N=8192, eta=0.2, alpha=1.5.

 T     K      Price_ref    Price_FFT     |e_P|        Delta_ref    Delta_FFT     |e_Δ|
----------------------------------------------------------------------------------------------------
0.25    90.0    10.712381    10.712639   2.59e-04       0.865118      0.865074   4.39e-05
0.25   100.0     3.987761     3.987761   2.06e-09       0.519939      0.519939   1.06e-09
0.25   110.0     0.953947     0.954178   2.30e-04       0.183236      0.183258   2.20e-05
1.00    90.0    13.589108    13.589299   1.91e-04       0.734606      0.734596   9.43e-06
1.00   100.0     7.965567     7.965567   4.48e-08       0.539828      0.539828   4.96e-09
1.00   110.0     4.292011     4.292154   1.43e-04       0.353254      0.353257   3.19e-06

Table: Gamma comparison (Reference vs FFT) in near–BS Heston
Ref: Nq=32768, umax=250.0, alpha=1.5.  FFT: N=8192, eta=0.2, alpha=1.5.

 T 



---


#Part 4
## Empirical data and hedging with FFT-Greeks on the real life data.



---



 Live European options (Deribit, USDC linear), USD-consistent FFT Greeks
Carr–Madan FFT + Lord–Kahl stable Heston CF
Use USDC-settled (linear) options -> standard Merton bounds apply
ATM implied vol -> near-BS anchor (not a calibration)
USD Greek sanity checks (bounds / monotone / convex)

In [9]:
# =========================================================
# Real Deribit options → Heston FFT prices & Greeks (USD)
# Robust no-arbitrage checks PER EXPIRY (incl. convexity).
# - Carr–Madan FFT (Simpson weights) + Lord–Kahl "little trap"
# - Prefer USDC-linear options; fallback to inverse (BTC-settled)
# - ATM BS IV → near-Black–Scholes Heston anchor (not a calibration)
# - Shape tests both at market strikes and on native FFT k-grid
# =========================================================
# pip install requests pandas numpy matplotlib

import numpy as np
import pandas as pd
import requests, time
from math import log, sqrt, exp, erf, pi
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# ---------------- Normal helpers ----------------
def _N(x):  # standard normal CDF
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def _phi(x):  # standard normal PDF
    return (1.0 / sqrt(2.0 * pi)) * np.exp(-0.5 * x * x)

# ---------------- Black–Scholes (USD) ----------------
def bs_price_delta_gamma_vega_v0(S0, K, T, r, q, sigma):
    if T <= 0:
        payoff = max(S0 - K, 0.0)
        return payoff, float(S0 > K), 0.0, 0.0
    sig = max(1e-12, sigma)
    d1 = (log(S0/K) + (r - q + 0.5*sig*sig)*T) / (sig * sqrt(T))
    d2 = d1 - sig*sqrt(T)
    disc_r = exp(-r*T); disc_q = exp(-q*T)
    price = S0*disc_q*_N(d1) - K*disc_r*_N(d2)
    delta = disc_q*_N(d1)
    gamma = disc_q*_phi(d1) / (S0 * sig * sqrt(T))
    vega_sigma = S0 * disc_q * sqrt(T) * _phi(d1)     # ∂C/∂σ
    vega_v0    = vega_sigma / (2.0 * sig)             # ∂C/∂v0 with v0=σ^2
    return float(price), float(delta), float(gamma), float(vega_v0)

def bs_implied_vol_call(price, S0, K, T, r, q, lo=1e-6, hi=5.0, tol=1e-8, maxit=100):
    # Robust bisection IV solver (bounds enforce arbitrage-consistent prices)
    if T <= 0 or S0 <= 0 or K <= 0:
        return 0.2
    disc_r = exp(-r*T); disc_q = exp(-q*T)
    lb = max(S0*disc_q - K*disc_r, 0.0); ub = S0*disc_q
    p = min(max(price, lb), ub)
    f_lo = bs_price_delta_gamma_vega_v0(S0, K, T, r, q, lo)[0] - p
    f_hi = bs_price_delta_gamma_vega_v0(S0, K, T, r, q, hi)[0] - p
    if f_lo*f_hi > 0:
        return 0.2
    a, b = lo, hi
    for _ in range(maxit):
        c = 0.5*(a+b)
        fc = bs_price_delta_gamma_vega_v0(S0, K, T, r, q, c)[0] - p
        if abs(fc) < tol or (b-a) < tol:
            return float(max(c, 1e-6))
        if f_lo*fc <= 0:
            b, f_hi = c, fc
        else:
            a, f_lo = c, fc
    return float(max(0.5*(a+b), 1e-6))

# ---------------- Heston CF (Lord–Kahl “little trap”) ----------------
# Ref: Lord & Kahl (2010), Mathematical Finance
def heston_charfunc_lord_kahl(u, T, v0, kappa, theta, sigma, rho, r, q, return_CD=False):
    iu = 1j * u
    a  = kappa * theta
    b  = kappa - rho * sigma * iu
    d  = np.sqrt(b*b + (iu + u*u) * sigma * sigma)
    g  = (b - d) / (b + d)
    e  = np.exp(-d * T)
    C  = (r - q) * iu * T + (a / (sigma*sigma)) * ((b - d) * T - 2.0 * np.log((1 - g*e) / (1 - g)))
    D  = ((b - d) / (sigma*sigma)) * ((1 - e) / (1 - g*e))
    phi = np.exp(C + D * v0)
    if return_CD: return phi, C, D
    return phi

# ---------------- Carr–Madan FFT (Simpson weights) ----------------
# Ref: Carr & Madan (1999), J. Computational Finance
def cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                    alpha=2.0, N=2**16, eta=0.08, k0=None):
    if T <= 0:
        raise ValueError("Use T>0 for transform-based pricing.")
    m = np.arange(N, dtype=float)
    u = m * eta
    z = u - 1j * (alpha + 1.0)

    phi, C, D = heston_charfunc_lord_kahl(z, T, v0, kappa, theta, sigma, rho, r, q, return_CD=True)
    denom = (alpha**2 + alpha - u*u) + 1j*(2*alpha + 1)*u
    base  = np.exp(-r*T) * (np.exp(1j * z * np.log(S0)) * phi) / denom

    Fp = base
    Fd = base * (1j * z / S0)
    Fg = base * (-(1j * z + z*z) / (S0*S0))
    Fv = base * D

    # Simpson weights (significantly reduces ringing vs trapezoid)
    w = np.ones(N)
    if N >= 3:
        w[1:-1:2] = 4.0
        w[2:-1:2] = 2.0
    w *= (eta/3.0)

    # Strike grid: exact centering at log S0
    dk = 2.0*np.pi/(N*eta)
    if k0 is None:
        k0 = np.log(S0) - 0.5*N*dk
    k  = k0 + np.arange(N) * dk
    kern = np.exp(-1j * u * k0) * w / np.pi

    P  = np.fft.fft(Fp * kern)
    Df = np.fft.fft(Fd * kern)
    Gf = np.fft.fft(Fg * kern)
    Vf = np.fft.fft(Fv  * kern)

    damp = np.exp(-alpha * k)
    return k, np.real(damp*P), np.real(damp*Df), np.real(damp*Gf), np.real(damp*Vf)

def price_greeks_fft_atK(S0, K, T, r, q, v0, kappa, theta, sigma, rho,
                         alpha=2.0, N=2**16, eta=0.08, k0=None):
    k_grid, P, D, G, V = cm_fft_grid_all(S0, T, r, q, v0, kappa, theta, sigma, rho,
                                         alpha=alpha, N=N, eta=eta, k0=k0)
    k = np.log(K)
    if k <= k_grid[0]:
        return float(P[0]), float(D[0]), float(G[0]), float(V[0])
    if k >= k_grid[-1]:
        return float(P[-1]), float(D[-1]), float(G[-1]), float(V[-1])
    idx = np.searchsorted(k_grid, k) - 1
    t = (k - k_grid[idx]) / (k_grid[idx+1] - k_grid[idx])
    price = (1-t)*P[idx] + t*P[idx+1]
    delta = (1-t)*D[idx] + t*D[idx+1]
    gamma = (1-t)*G[idx] + t*G[idx+1]
    vega  = (1-t)*V[idx] + t*V[idx+1]
    return float(price), float(delta), float(gamma), float(vega)

# ------------- Near-Black–Scholes anchor (not a calibration) -------------
def heston_params_near_bs(sigma_atm):
    return {
        "v0":    float(sigma_atm**2),
        "theta": float(sigma_atm**2),
        "kappa": 100.0,    # avoids stiffness; still near-BS
        "sigma": 1e-3,     # small vol-of-vol
        "rho":   0.0
    }

# ---------------- Deribit helpers ----------------
DERIBIT = "https://www.deribit.com/api/v2"

def get_json(endpoint, params):
    r = requests.get(DERIBIT + endpoint, params=params, timeout=10)
    r.raise_for_status()
    j = r.json()
    if "result" not in j:
        raise RuntimeError(f"Unexpected response for {endpoint}: {j}")
    return j["result"]

def list_linear_btc_options():
    ins = get_json("/public/get_instruments", {
        "currency": "USDC", "kind": "option", "expired": "false"
    })
    return [it for it in ins
            if it.get("base_currency") == "BTC"
            and (it.get("settlement_currency") == "USDC" or it.get("counter_currency") == "USDC")]

def list_inverse_btc_options():
    ins = get_json("/public/get_instruments", {
        "currency": "BTC", "kind": "option", "expired": "false"
    })
    return [it for it in ins
            if it.get("base_currency") == "BTC"
            and it.get("settlement_currency") in ("BTC", None)]

def get_mid_from_ticker(name):
    tk = get_json("/public/ticker", {"instrument_name": name})
    b = tk.get("best_bid_price") or tk.get("bid_price")
    a = tk.get("best_ask_price") or tk.get("ask_price")
    mid = None
    if b and a and a > b:
        mid = 0.5 * (b + a)
    elif tk.get("mark_price"):
        mid = float(tk["mark_price"])
    return mid, b, a

# ---------- Build slices (USDC-first, inverse fallback), collect M expiries ----------
def build_slice_linear(instruments, now_ms, min_pts=8):
    expiries = sorted({it["expiration_timestamp"] for it in instruments if it["expiration_timestamp"] > now_ms})
    buckets = []
    for exp_ts in expiries:
        calls = [it for it in instruments
                 if it["expiration_timestamp"] == exp_ts and it["option_type"] == "call"]
        rows = []
        for it in calls:
            name = it["instrument_name"]
            mid_usd, b, a = get_mid_from_ticker(name)  # linear → already USD
            if mid_usd is None or mid_usd <= 0: continue
            K = float(it["strike"])
            T = max(0.0, (exp_ts - now_ms) / (365.0*24*3600*1000.0))
            rows.append({"expiry": exp_ts, "instrument": name, "K": K, "T": T,
                         "mid_btc": np.nan, "mid_usd": float(mid_usd)})
        if len(rows) >= min_pts:
            buckets.append(rows)
    return buckets

def build_slice_inverse(instruments, now_ms, S0, min_pts=10):
    expiries = sorted({it["expiration_timestamp"] for it in instruments if it["expiration_timestamp"] > now_ms})
    buckets = []
    for exp_ts in expiries:
        calls = [it for it in instruments
                 if it["expiration_timestamp"] == exp_ts and it["option_type"] == "call"]
        rows = []
        for it in calls:
            name = it["instrument_name"]
            mid_btc, b, a = get_mid_from_ticker(name)
            if mid_btc is None or mid_btc <= 0: continue
            K = float(it["strike"])
            T = max(0.0, (exp_ts - now_ms) / (365.0*24*3600*1000.0))
            mid_usd = float(mid_btc) * S0
            rows.append({"expiry": exp_ts, "instrument": name, "K": K, "T": T,
                         "mid_btc": float(mid_btc), "mid_usd": mid_usd})
        if len(rows) >= min_pts:
            buckets.append(rows)
    return buckets

# ---------------- Shape checks (per expiry) ----------------
def shape_checks_per_expiry(df_out, rel_tol=1e-3, abs_tol=2e-4):
    """Return summary row per expiry using robust, scale-aware tolerances."""
    res = []
    for exp_ts, grp in df_out.groupby("expiry"):
        g = grp.sort_values("K").reset_index(drop=True).copy()
        price = g["P_fft_usd"].to_numpy()

        # Monotonicity at market strikes
        mono_raw  = np.diff(price)
        mono_thr  = np.maximum(abs_tol, rel_tol*np.maximum(price[:-1], price[1:]))
        mono_viol = np.maximum(0.0, mono_raw - mono_thr)

        # Convexity at market strikes (discrete second diff)
        if len(price) >= 3:
            sec   = price[:-2] - 2.0*price[1:-1] + price[2:]
            local = np.maximum.reduce([price[:-2], price[1:-1], price[2:]])
            thr   = np.maximum(abs_tol, rel_tol*local)
            conv_viol = np.maximum(0.0, -(sec) - thr)
        else:
            conv_viol = np.array([])

        res.append({
            "expiry_utc": datetime.fromtimestamp(exp_ts/1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M"),
            "T_days": float((g["T"].iloc[0]*365.0)),
            "n_strikes": int(len(g)),
            "merton_bounds_ok": bool(g["ok_bounds"].all()),
            "delta_bounds_ok":  bool(g["ok_delta"].all()),
            "gamma_nonneg_ok":  bool(g["ok_gamma"].all()),
            "vega_v0_nonneg_ok":bool(g["ok_vega"].all()),
            "mono_viol_cnt": int((mono_viol > 0).sum()),
            "conv_viol_cnt": int((conv_viol > 0).sum()),
            "mono_viol_max": float(mono_viol.max() if mono_viol.size else 0.0),
            "conv_viol_max": float(conv_viol.max() if conv_viol.size else 0.0),
        })
    return pd.DataFrame(res)

# =========================================================
# Main
# =========================================================
if __name__ == "__main__":
    now_ms = int(time.time()*1000)

    # Underlying (USD)
    idx = get_json("/public/get_index_price", {"index_name": "btc_usd"})
    S0 = float(idx.get("index_price", idx.get("btc")))
    r, q = 0.02, 0.0

    # Gather up to M expiries (linear first, then inverse)
    M = 3  # how many expiries to analyze
    buckets = build_slice_linear(list_linear_btc_options(), now_ms, min_pts=8)
    if not buckets:
        buckets = build_slice_inverse(list_inverse_btc_options(), now_ms, S0, min_pts=10)
    if not buckets:
        raise SystemExit("No liquid call slices found (linear nor inverse). Try later.")

    buckets = buckets[:M]

    all_rows = []

    for rows in buckets:
        df = pd.DataFrame(rows).sort_values("K").reset_index(drop=True)

        # ATM IV anchor
        atm_row = df.iloc[(df["K"] - S0).abs().argmin()]
        sigma_atm = bs_implied_vol_call(atm_row["mid_usd"], S0,
                                        float(atm_row["K"]), float(atm_row["T"]), r, q)
        hp = heston_params_near_bs(sigma_atm)

        # FFT params (tight): Simpson weights are inside cm_fft_grid_all
        ALPHA = 2.0
        N     = 2**16
        ETA   = 0.08
        dk    = 2.0*np.pi/(N*ETA)
        k0    = np.log(S0) - 0.5*N*dk  # exact centering

        # Compute FFT prices/Greeks at market strikes
        def fft_tuple(row):
            P, D, G, Vv0 = price_greeks_fft_atK(
                S0, float(row.K), float(row.T), r, q,
                hp["v0"], hp["kappa"], hp["theta"], hp["sigma"], hp["rho"],
                alpha=ALPHA, N=N, eta=ETA, k0=k0
            )
            return float(P), float(D), float(G), float(Vv0)

        greeks = [fft_tuple(row) for row in df.itertuples(index=False)]
        gdf = pd.DataFrame(greeks, columns=["P_fft_usd","Delta_usd","Gamma_usd","Vega_v0_usd"])
        out = pd.concat([df.reset_index(drop=True), gdf], axis=1)

        # --- Merton/Greeks sanity bounds (USD) ---
        lower = np.maximum(S0*np.exp(-q*out["T"]) - out["K"]*np.exp(-r*out["T"]), 0.0)
        upper = S0*np.exp(-q*out["T"])
        tolPrice = 1e-6 * S0 + 2e-3
        ok_bounds = (out["P_fft_usd"] >= lower - tolPrice) & (out["P_fft_usd"] <= upper + tolPrice)
        tolUnit = 1e-3
        ok_delta = (out["Delta_usd"] >= -tolUnit) & (out["Delta_usd"] <= np.exp(-q*out["T"]) + tolUnit)
        tolGamma = 5e-6
        ok_gamma = (out["Gamma_usd"] >= -tolGamma)
        ok_vega  = (out["Vega_v0_usd"] >= -1e-3)

        out = out.assign(ok_bounds=ok_bounds, ok_delta=ok_delta, ok_gamma=ok_gamma, ok_vega=ok_vega)

        # Print brief table for this expiry
        print("\n=== Expiry:", datetime.fromtimestamp(int(out['expiry'].iloc[0])/1000, tz=timezone.utc),
              f"(rows={len(out)}) | ATM σ ~ {sigma_atm:.4f}")
        show = out.copy()
        show["T_days"] = (show["T"] * 365.0).round(2)
        cols = ["instrument","K","T_days","mid_usd","P_fft_usd","Delta_usd","Gamma_usd","Vega_v0_usd",
                "ok_bounds","ok_delta","ok_gamma","ok_vega"]
        print(show[cols].to_string(index=False, float_format=lambda x: f"{x:,.6g}"))

        all_rows.append(out)

    # Concatenate expiries and run shape checks per expiry
    big = pd.concat(all_rows, ignore_index=True)
    shape_df = shape_checks_per_expiry(big, rel_tol=1e-3, abs_tol=2e-4)
    print("\n--- Shape checks per expiry (market strikes) ---")
    print(shape_df.to_string(index=False, float_format=lambda x: f"{x:,.6g}"))

    # Slice-level pass/fail bars
    labels = ["Merton bounds", "Delta bounds", "Gamma ≥ 0", r"Vega$_{v0}$ ≥ 0", "Monotone in K", "Convex in K"]
    pass_per_expiry = []
    for _, row in shape_df.iterrows():
        flags = [
            bool(row["merton_bounds_ok"]),
            bool(row["delta_bounds_ok"]),
            bool(row["gamma_nonneg_ok"]),
            bool(row["vega_v0_nonneg_ok"]),
            row["mono_viol_cnt"] == 0,
            row["conv_viol_cnt"] == 0,
        ]
        pass_per_expiry.append(flags)
    pass_per_expiry = np.array(pass_per_expiry, dtype=bool)

    # Count how many expiries pass each test
    pass_counts = pass_per_expiry.sum(axis=0)
    fail_counts = len(shape_df) - pass_counts

    x = np.arange(len(labels))
    plt.figure()
    plt.bar(x - 0.2, pass_counts, width=0.4, label="Pass")
    plt.bar(x + 0.2, fail_counts, width=0.4, label="Fail")
    plt.xticks(x, labels, rotation=20)
    plt.ylabel("Number of expiries (pass/fail)")
    plt.title("Sanity & shape checks: pass/fail (per expiry)")
    plt.legend()
    plt.tight_layout()
    plt.savefig("bars_checks_pass_fail.png", dpi=150)
    plt.close()



=== Expiry: 2025-09-14 08:00:00+00:00 (rows=16) | ATM σ ~ 0.1517
               instrument       K  T_days  mid_usd    P_fft_usd   Delta_usd    Gamma_usd  Vega_v0_usd  ok_bounds  ok_delta  ok_gamma  ok_vega
BTC_USDC-14SEP25-104000-C 104,000    0.58 11,641.7     11,626.8           1 -4.73484e-14 -2.85368e-06       True      True      True     True
BTC_USDC-14SEP25-106000-C 106,000    0.58 9,642.12     9,626.88           1  2.49811e-14 -2.20047e-06       True      True      True     True
BTC_USDC-14SEP25-108000-C 108,000    0.58 7,642.56     7,626.96           1  6.65914e-15 -2.44545e-06       True      True      True     True
BTC_USDC-14SEP25-109000-C 109,000    0.58 6,642.93     6,626.99           1 -8.07874e-15 -2.60583e-06       True      True      True     True
BTC_USDC-14SEP25-110000-C 110,000    0.58 5,643.12     5,627.01           1 -7.10779e-15 -2.59993e-06       True      True      True     True
BTC_USDC-14SEP25-111000-C 111,000    0.58 4,643.93     4,627.06           1  8.061